<a href="https://colab.research.google.com/github/aditya4131/csc4343-homework1/blob/main/4343homework1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import requests
from pathlib import Path

'''
You need to implement:

class ResSect(nn.Module):
    def __init__(self, n_filter, n_residual_blocks, beginning_stride):
        Initialize the sector by creating layers needed
        n_filter: number of filters in the conv layers in the blocks
        n_residual_blocks: number of blocks in this sector
        beginning_stride: the stride of the first conv of the first block in the sector

    def forward(self, x):
        Implement computation performed in the sector
        x: input tensor
        You should return the result tensor
'''

class ResSect(nn.Module):
    def __init__(self, in_channels, n_residual_blocks, beginning_stride):
        super().__init__()

        strides = [beginning_stride] + [1] * (n_residual_blocks - 1)
        out_channels = in_channels
        if in_channels == 64 or in_channels == 128:
            in_channels = in_channels // 2
        layers = []
        for stride in strides:
            print(in_channels, out_channels, beginning_stride)
            layers.append(ResBlock(in_channels, out_channels, beginning_stride))
            in_channels = out_channels
        self.blocks = nn.Sequential(*layers)

    def forward(self, x):
        return self.blocks(x)

class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, beginning_stride=1):
        super().__init__()
        self.conv1_layer = nn.Sequential(
            nn.Conv2d(in_channels,
                      out_channels,
                      kernel_size=3,
                      padding=1,
                      stride=beginning_stride),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )
        self.conv2_layer = nn.Sequential(
            nn.Conv2d(out_channels,
                      out_channels,
                      kernel_size=3,
                      padding=1,
                      stride=1),
            nn.BatchNorm2d(out_channels)
        )

        self.relu_layer = nn.ReLU()

        self.downsample_layer = nn.Sequential()
        if beginning_stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1,
                          stride=beginning_stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = x
        x = self.conv1_layer(x)
        x = self.conv2_layer(x)
        x += self.downsample_layer(residual)
        x = self.relu_layer(x)
        return x

class ResModel(nn.Module):
    def __init__(self, pretrained=False):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU()

        self.sect1 = ResSect(32, 3, 1)
        self.sect2 = ResSect(64, 3, 2)
        self.sect3 = ResSect(128, 3, 2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(128, 10)

        if pretrained:
            self.load_trained_model()

    def load_trained_model(self):
        '''
        You need to implement this function to:
            1. download the saved pretrained model from your online location
            2. load model from the downloaded model file
        '''

        print("Downloading")
        request = requests.get("https://github.com/aditya4131/csc4343-homework1/raw/main/model-3.pth")
        with open("model-3.pth", "wb") as f:
            f.write(request.content)

        return self.load_state_dict(torch.load("model-3.pth"))

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.sect1(x)
        x = self.sect2(x)
        x = self.sect3(x)

        x = self.avgpool(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

device = 'cuda'


In [25]:
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 128

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [26]:
model = ResModel().to(device)

In [27]:
def accuracy_fn(y_true, y_pred):
  correct = torch.eq(y_true, y_pred).sum().item()
  acc = (correct/len(y_pred)) * 100
  return acc


def train_step(model:torch.nn.Module,
               dataloader:torch.utils.data.DataLoader,
               loss_fn:torch.nn.Module,
               optimizer:torch.optim.Optimizer,
               accuracy_fn,
               device:torch.device):
  train_loss, train_acc = 0,0
  model.train()

  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    train_loss += loss
    train_acc += accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1))
    optimizer.zero_grad()
    # loss.requires_grad = True
    loss.backward()
    optimizer.step()

  train_loss /= len(dataloader)
  train_acc /= len(dataloader)

  print(f"Train loss: {train_loss} | Train accuracy: {train_acc}")

def test_step(model:torch.nn.Module,
              dataloader:torch.utils.data.DataLoader,
              loss_fn:torch.nn.Module,
              accuracy_fn,
              device:torch.device):
  test_loss, test_acc = 0,0

  with torch.inference_mode():
    for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)
      y_pred = model(X)
      loss = loss_fn(y_pred, y)
      test_loss += loss
      test_acc += accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1))
      optimizer.zero_grad()
      loss.requires_grad = True
      loss.backward()
      optimizer.step()

    test_loss /= len(dataloader)
    test_acc /= len(dataloader)

    print(f"Train loss: {test_loss} | Train accuracy: {test_acc}")

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lr=0.00001, params=model.parameters())


In [28]:
epochs = 70

for epoch in range(epochs):
  train_step(model=model,
             dataloader=trainloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             accuracy_fn=accuracy_fn,
             device=device)
  test_step(model=model,
             dataloader=trainloader,
             loss_fn=loss_fn,
             accuracy_fn=accuracy_fn,
             device=device)

Train loss: 2.0007381439208984 | Train accuracy: 26.973305626598467
Train loss: 1.7119922637939453 | Train accuracy: 37.71539322250639
Train loss: 1.6071504354476929 | Train accuracy: 41.47818094629156
Train loss: 1.5085219144821167 | Train accuracy: 44.790201406649615
Train loss: 1.4498095512390137 | Train accuracy: 47.25863171355499
Train loss: 1.3823585510253906 | Train accuracy: 50.0147858056266
Train loss: 1.3465465307235718 | Train accuracy: 51.283168158567776
Train loss: 1.2933015823364258 | Train accuracy: 53.33639705882353
Train loss: 1.2688267230987549 | Train accuracy: 54.41496163682864
Train loss: 1.2201076745986938 | Train accuracy: 56.25439578005115
Train loss: 1.2062830924987793 | Train accuracy: 56.755514705882355
Train loss: 1.167755126953125 | Train accuracy: 58.161365089514064
Train loss: 1.1544277667999268 | Train accuracy: 58.77038043478261
Train loss: 1.1196249723434448 | Train accuracy: 60.15265345268542
Train loss: 1.1105080842971802 | Train accuracy: 60.5278932

In [30]:
model.to(torch.device('cpu'))
torch.save(model.state_dict(), './model.pth')